# Lab 06 (심화) — Classification: Advanced Topics


> **전제 조건:** Lab 06 기본편 완료 (시그모이드, 소프트맥스, 기본 평가 지표)

---

## 학습 목표

| # | 목표 | 예상 시간 |
|---|---|---|
| 1 | Dropout · BatchNorm — 정규화로 과적합 방지 | 15분 |
| 2 | 클래스 불균형 — Weighted Loss · 오버샘플링 | 15분 |
| 3 | 학습률 스케줄러 · 조기 종료 | 15분 |
| 4 | 다중 클래스 ROC · Label Smoothing | 10분 |
| 5 | Exercise | 5분 |

---

**데이터셋:** MNIST — 손글씨 숫자 분류 (불균형 버전 포함)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, WeightedRandomSampler
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, f1_score, confusion_matrix,
    roc_curve, roc_auc_score
)

# 한글 폰트
_fp = '/System/Library/Fonts/AppleGothic.ttf'
fm.fontManager.addfont(_fp)
plt.rcParams['font.family'] = fm.FontProperties(fname=_fp).get_name()
plt.rcParams['axes.unicode_minus'] = False
sns.set_theme(style='whitegrid')

torch.manual_seed(42)
np.random.seed(42)
print('PyTorch:', torch.__version__)

In [ ]:
# ── MNIST 로드 (기본편과 동일) ─────────────────────────────────────────
try:
    from torchvision import datasets, transforms
    mnist_tr_full = datasets.MNIST(root='/tmp/mnist', train=True,  download=True,
                                   transform=transforms.ToTensor())
    mnist_te_full = datasets.MNIST(root='/tmp/mnist', train=False, download=True,
                                   transform=transforms.ToTensor())
    X_tr_raw = mnist_tr_full.data.float().view(-1, 784) / 255.0
    y_tr_raw = mnist_tr_full.targets
    X_te_raw = mnist_te_full.data.float().view(-1, 784) / 255.0
    y_te_raw = mnist_te_full.targets
except ImportError:
    from sklearn.datasets import fetch_openml
    print('torchvision 없음 → sklearn MNIST')
    _d = fetch_openml('mnist_784', version=1, as_frame=False, parser='auto')
    _X = torch.tensor(_d.data.astype(np.float32) / 255.0)
    _y = torch.tensor(_d.target.astype(np.int64))
    X_tr_raw, X_te_raw = _X[:60000], _X[60000:]
    y_tr_raw, y_te_raw = _y[:60000], _y[60000:]

# 실습용 서브셋
X_mn_tr = X_tr_raw[:6000]
y_mn_tr = y_tr_raw[:6000]
X_mn_te = X_te_raw[:1000]
y_mn_te = y_te_raw[:1000]

print(f'Train: {X_mn_tr.shape}  Test: {X_mn_te.shape}')

# 공통 DataLoader 생성 함수
def make_loader(X, y, batch_size=256, shuffle=True, sampler=None):
    ds = TensorDataset(X, y)
    return DataLoader(ds, batch_size=batch_size, shuffle=(shuffle and sampler is None),
                      sampler=sampler)

train_loader = make_loader(X_mn_tr, y_mn_tr)
test_loader  = make_loader(X_mn_te, y_mn_te, shuffle=False)

# 공통 평가 함수
@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    total_loss, correct, n = 0, 0, 0
    criterion = nn.CrossEntropyLoss()
    for xb, yb in loader:
        out  = model(xb)
        total_loss += criterion(out, yb).item() * len(xb)
        correct    += (out.argmax(1) == yb).sum().item()
        n          += len(xb)
    return total_loss / n, correct / n

print('데이터 준비 완료')

---
## Part 1. 정규화 기법 — Dropout & BatchNorm

### 1-1. Dropout

**Dropout**은 학습 중 뉴런을 무작위로 비활성화해 과적합을 방지합니다.

```
학습 시:  [0.3, 0.0, 0.8, 0.0, 0.5]  ← 뉴런 2개를 0으로
추론 시:  [0.3, 0.4, 0.8, 0.1, 0.5]  ← 모든 뉴런 활성화 (스케일 보정)
```

**핵심 주의 사항:**
- 학습: `model.train()` → Dropout **활성화**
- 추론: `model.eval()` → Dropout **비활성화**
- PyTorch는 `1/(1-p)` 스케일링을 자동 처리 (inverted dropout)

In [ ]:
# Dropout 동작 원리 시각화
torch.manual_seed(0)
x_demo = torch.ones(1, 10) * 2.0   # 모든 값이 2인 벡터

drop_layer = nn.Dropout(p=0.5)

print('입력:', x_demo.numpy())
print()
drop_layer.train()
for trial in range(5):
    out = drop_layer(x_demo)
    print(f'학습 모드 시도{trial+1}: {out.numpy()}  (합={out.sum():.1f})')
print()
drop_layer.eval()
out_eval = drop_layer(x_demo)
print(f'추론 모드      : {out_eval.numpy()}  (합={out_eval.sum():.1f})')
print('→ eval 모드에서는 입력이 그대로 통과 (p=0으로 동작)')
print('→ train 모드에서 1/(1-0.5)=2배 스케일링으로 기댓값 보존')

# Dropout 비율별 활성화 패턴 시각화
fig, axes = plt.subplots(1, 4, figsize=(14, 3))
x_vis = torch.randn(20, 20)
for ax, p in zip(axes, [0.0, 0.2, 0.5, 0.8]):
    d = nn.Dropout(p=p)
    d.train()
    mask = (d(torch.ones_like(x_vis)) > 0).float()
    ax.imshow(mask, cmap='Blues', vmin=0, vmax=1)
    ax.set_title(f'Dropout p={p}\n활성 뉴런: {mask.mean():.0%}')
    ax.axis('off')
plt.suptitle('Dropout 비율별 뉴런 활성화 패턴 (흰=비활성, 파랑=활성)', y=1.04)
plt.tight_layout()
plt.show()

### 1-2. Batch Normalization

**BatchNorm**은 미니배치 내에서 활성화값을 정규화합니다:

$$\hat{x}_i = \frac{x_i - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}}, \quad y_i = \gamma\hat{x}_i + \beta$$

- $\mu_B, \sigma_B$: 배치의 평균·표준편차 (학습 시 계산)
- $\gamma, \beta$: 학습 가능한 스케일·시프트 파라미터

**효과:**
- Internal Covariate Shift 감소 → 더 높은 학습률 사용 가능
- 약한 정규화 효과 (Dropout의 대안)

In [ ]:
# BatchNorm 효과: 레이어 출력 분포 변화 시각화
torch.manual_seed(42)
x_bn = torch.randn(256, 128) * 5 + 3   # 평균 3, 표준편차 5

bn_layer = nn.BatchNorm1d(128)
bn_layer.eval()
x_bn_out = bn_layer(x_bn).detach()

fig, axes = plt.subplots(1, 3, figsize=(13, 3.5))
axes[0].hist(x_bn[:, 0].numpy(), bins=40, color='tomato', edgecolor='white', alpha=0.85)
axes[0].set_title(f'BatchNorm 전\n(μ={x_bn[:,0].mean():.2f}, σ={x_bn[:,0].std():.2f})')
axes[0].set_xlabel('활성화 값')

axes[1].hist(x_bn_out[:, 0].numpy(), bins=40, color='steelblue', edgecolor='white', alpha=0.85)
axes[1].set_title(f'BatchNorm 후\n(μ={x_bn_out[:,0].mean():.2f}, σ={x_bn_out[:,0].std():.2f})')
axes[1].set_xlabel('활성화 값')

# 100개 특성의 평균/분산 분포
axes[2].scatter(x_bn.mean(0).numpy(), x_bn.std(0).numpy(),
                color='tomato', s=20, alpha=0.6, label='BN 전')
axes[2].scatter(x_bn_out.mean(0).numpy(), x_bn_out.std(0).numpy(),
                color='steelblue', s=20, alpha=0.6, label='BN 후')
axes[2].set_title('특성별 평균 vs 표준편차\n(BN 후 모든 특성이 μ≈0, σ≈1에 집중)')
axes[2].set_xlabel('평균'); axes[2].set_ylabel('표준편차')
axes[2].legend()
plt.suptitle('Batch Normalization 효과', y=1.04)
plt.tight_layout()
plt.show()

### 1-3. 정규화 기법 조합 비교 — MNIST

In [ ]:
def make_classifier(use_dropout=False, use_bn=False, p_drop=0.3):
    """정규화 조합별 MNIST 분류기 생성"""
    layers = []
    in_dim = 784
    for out_dim in [256, 128, 64]:
        layers.append(nn.Linear(in_dim, out_dim))
        if use_bn:
            layers.append(nn.BatchNorm1d(out_dim))
        layers.append(nn.ReLU())
        if use_dropout:
            layers.append(nn.Dropout(p=p_drop))
        in_dim = out_dim
    layers.append(nn.Linear(in_dim, 10))
    return nn.Sequential(*layers)

def train_model(model, epochs=30, lr=0.001):
    opt  = torch.optim.Adam(model.parameters(), lr=lr)
    crit = nn.CrossEntropyLoss()
    tr_losses, te_accs = [], []
    for _ in range(epochs):
        model.train()
        epoch_loss = 0
        for xb, yb in train_loader:
            opt.zero_grad()
            loss = crit(model(xb), yb)
            loss.backward()
            opt.step()
            epoch_loss += loss.item() * len(xb)
        tr_losses.append(epoch_loss / len(train_loader.dataset))
        _, te_acc = evaluate(model, test_loader)
        te_accs.append(te_acc)
    return tr_losses, te_accs

configs = [
    ('기본 (정규화 없음)',    dict(use_dropout=False, use_bn=False)),
    ('Dropout (p=0.3)',       dict(use_dropout=True,  use_bn=False, p_drop=0.3)),
    ('BatchNorm',             dict(use_dropout=False, use_bn=True)),
    ('Dropout + BatchNorm',   dict(use_dropout=True,  use_bn=True,  p_drop=0.3)),
]
colors_cfg = ['gray', 'tomato', 'steelblue', 'seagreen']

all_results = {}
for name, cfg in configs:
    torch.manual_seed(42)
    m = make_classifier(**cfg)
    tr_l, te_a = train_model(m, epochs=30)
    all_results[name] = dict(model=m, tr_losses=tr_l, te_accs=te_a)
    print(f'{name:<25} 최종 Test Acc: {te_a[-1]:.4f}')

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(13, 4))
for (name, res), color in zip(all_results.items(), colors_cfg):
    axes[0].plot(res['tr_losses'], color=color, lw=2, label=name)
    axes[1].plot(res['te_accs'],   color=color, lw=2, label=name)

axes[0].set_title('학습 손실 비교')
axes[0].set_xlabel('Epoch'); axes[0].set_ylabel('Train Loss')
axes[0].legend(fontsize=8)

axes[1].set_title('Test Accuracy 비교')
axes[1].set_xlabel('Epoch'); axes[1].set_ylabel('Test Accuracy')
axes[1].set_ylim(0.7, 1.0)
axes[1].legend(fontsize=8)

plt.suptitle('정규화 기법 조합별 MNIST 학습 비교', y=1.02)
plt.tight_layout()
plt.show()

# 과적합 정도 측정: 마지막 에폭 Train Loss vs Test Loss
print(f'{'방법':<25} {'Train Loss':>12} {'Test Acc':>10}')
print('-' * 50)
for name, res in all_results.items():
    _, tr_acc = evaluate(res['model'], train_loader)
    _, te_acc = evaluate(res['model'], test_loader)
    gap = tr_acc - te_acc
    print(f'{name:<25} {res["tr_losses"][-1]:>12.4f} {te_acc:>10.4f}  (Train-Test Gap={gap:+.4f})')

---
## Part 2. 클래스 불균형 처리

### 2-1. 불균형 데이터의 문제

실제 데이터는 클래스 간 샘플 수 차이가 큰 경우가 많습니다:
- 의료: 정상 95% vs 질환 5%
- 금융: 정상 거래 99% vs 사기 1%

**정확도 역설 (Accuracy Paradox):**  
소수 클래스를 무조건 다수 클래스로 예측해도 높은 정확도!

$$\text{Acc} = \frac{9500}{10000} = 95\% \quad (\text{하지만 질환을 하나도 못 찾음})$$

In [ ]:
# 불균형 MNIST 데이터 생성 (클래스 0~4: 전체, 클래스 5~9: 10%만)
def make_imbalanced(X, y, minority_ratio=0.1, minority_classes=None):
    if minority_classes is None:
        minority_classes = list(range(5, 10))
    keep_mask = torch.zeros(len(y), dtype=torch.bool)
    for c in range(10):
        idx = (y == c).nonzero(as_tuple=True)[0]
        if c in minority_classes:
            n_keep = max(1, int(len(idx) * minority_ratio))
            perm   = torch.randperm(len(idx))[:n_keep]
            keep_mask[idx[perm]] = True
        else:
            keep_mask[idx] = True
    return X[keep_mask], y[keep_mask]

torch.manual_seed(0)
X_imb, y_imb = make_imbalanced(X_mn_tr, y_mn_tr, minority_ratio=0.1)

# 클래스 분포 시각화
counts_balanced   = torch.bincount(y_mn_tr)
counts_imbalanced = torch.bincount(y_imb)

fig, axes = plt.subplots(1, 2, figsize=(13, 3.5))
x_pos = np.arange(10)
colors_bal = ['steelblue']*5 + ['tomato']*5
axes[0].bar(x_pos, counts_balanced.numpy(), color='steelblue', edgecolor='k', alpha=0.85)
axes[0].set_title('균형 데이터 — 클래스별 샘플 수')
axes[0].set_xlabel('숫자 클래스'); axes[0].set_ylabel('샘플 수')
for i, v in enumerate(counts_balanced):
    axes[0].text(i, v+5, str(v.item()), ha='center', fontsize=8)

axes[1].bar(x_pos, counts_imbalanced.numpy(), color=colors_bal, edgecolor='k', alpha=0.85)
axes[1].set_title('불균형 데이터 — 클래스 5~9가 10%')
axes[1].set_xlabel('숫자 클래스'); axes[1].set_ylabel('샘플 수')
for i, v in enumerate(counts_imbalanced):
    axes[1].text(i, v+2, str(v.item()), ha='center', fontsize=8)
handles = [plt.Rectangle((0,0),1,1, color='steelblue'), plt.Rectangle((0,0),1,1, color='tomato')]
axes[1].legend(handles, ['다수 클래스(0~4)', '소수 클래스(5~9)'])

plt.tight_layout()
plt.show()

print(f'균형 데이터   총 {len(y_mn_tr)}개, 클래스당 ~{len(y_mn_tr)//10}개')
print(f'불균형 데이터 총 {len(y_imb)}개, 0~4: ~{counts_imbalanced[:5].float().mean():.0f}개, '
      f'5~9: ~{counts_imbalanced[5:].float().mean():.0f}개')

### 2-2. 해결책 비교

| 방법 | 원리 | 특징 |
|---|---|---|
| **기본 (처리 없음)** | 그대로 학습 | 소수 클래스 무시 |
| **Weighted Loss** | 소수 클래스 손실에 가중치 | 구현 쉬움, 안정적 |
| **WeightedRandomSampler** | 소수 클래스를 더 자주 샘플링 | 에폭당 다양성↑ |

**클래스 가중치 계산:**  
$$w_c = \frac{N}{K \cdot n_c}$$
- $N$: 전체 샘플, $K$: 클래스 수, $n_c$: 클래스 $c$의 샘플 수

In [ ]:
# 클래스 가중치 계산
N, K = len(y_imb), 10
class_weights = torch.tensor(
    [N / (K * counts_imbalanced[c].item()) for c in range(10)],
    dtype=torch.float32
)
print('클래스 가중치:')
for c, w in enumerate(class_weights):
    print(f'  클래스 {c}: {w:.3f}')

# WeightedRandomSampler용 샘플 가중치
sample_weights = class_weights[y_imb]
sampler = WeightedRandomSampler(sample_weights, num_samples=len(y_imb), replacement=True)

# 데이터로더 3가지
loader_plain  = make_loader(X_imb, y_imb)
loader_sampler= make_loader(X_imb, y_imb, shuffle=False, sampler=sampler)

# Weighted Loss 모델 학습
crit_weighted = nn.CrossEntropyLoss(weight=class_weights)

def train_imb(loader, use_weighted_loss=False, epochs=25):
    torch.manual_seed(42)
    model = make_classifier(use_dropout=True, use_bn=True)
    opt   = torch.optim.Adam(model.parameters(), lr=0.001)
    crit  = crit_weighted if use_weighted_loss else nn.CrossEntropyLoss()
    te_accs, te_f1s = [], []
    for _ in range(epochs):
        model.train()
        for xb, yb in loader:
            opt.zero_grad()
            loss = crit(model(xb), yb)
            loss.backward()
            opt.step()
        model.eval()
        with torch.no_grad():
            preds = model(X_mn_te).argmax(1).numpy()
        te_accs.append(accuracy_score(y_mn_te.numpy(), preds))
        te_f1s.append(f1_score(y_mn_te.numpy(), preds, average='macro'))
    return model, te_accs, te_f1s

print('\n모델 학습 중...')
m_plain,   acc_plain,   f1_plain   = train_imb(loader_plain,   use_weighted_loss=False)
m_wloss,   acc_wloss,   f1_wloss   = train_imb(loader_plain,   use_weighted_loss=True)
m_sampler, acc_sampler, f1_sampler = train_imb(loader_sampler, use_weighted_loss=False)
print('완료')

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(13, 4))
labels_m = ['기본 (처리 없음)', 'Weighted Loss', 'WeightedSampler']
colors_m = ['gray', 'tomato', 'steelblue']

for (label, accs, f1s), color in zip(
    [(labels_m[0], acc_plain, f1_plain),
     (labels_m[1], acc_wloss, f1_wloss),
     (labels_m[2], acc_sampler, f1_sampler)], colors_m):
    axes[0].plot(accs, color=color, lw=2, label=label)
    axes[1].plot(f1s,  color=color, lw=2, label=label)

axes[0].set_title('Test Accuracy — 불균형 데이터')
axes[0].set_xlabel('Epoch'); axes[0].set_ylabel('Accuracy')
axes[0].set_ylim(0.5, 1.0); axes[0].legend(fontsize=9)

axes[1].set_title('Test Macro F1 — 불균형 데이터')
axes[1].set_xlabel('Epoch'); axes[1].set_ylabel('Macro F1')
axes[1].set_ylim(0.3, 1.0); axes[1].legend(fontsize=9)

plt.suptitle('클래스 불균형 처리 방법 비교', y=1.02)
plt.tight_layout()
plt.show()

# 클래스별 F1 비교
print(f'\n{'방법':<20} {'Acc':>8} {'MacroF1':>10}')
print('-' * 42)
for label, m in zip(labels_m, [m_plain, m_wloss, m_sampler]):
    m.eval()
    with torch.no_grad():
        preds = m(X_mn_te).argmax(1).numpy()
    acc = accuracy_score(y_mn_te.numpy(), preds)
    f1  = f1_score(y_mn_te.numpy(), preds, average='macro')
    f1s_per = f1_score(y_mn_te.numpy(), preds, average=None)
    print(f'{label:<20} {acc:>8.4f} {f1:>10.4f}')
    print(f'  클래스별 F1: {[f"{v:.2f}" for v in f1s_per]}')

print('\n→ Accuracy만 보면 "기본"이 좋아 보이지만, Macro F1은 소수 클래스 성능을 반영!')

---
## Part 3. 학습률 스케줄러와 조기 종료

### 3-1. 학습률 스케줄러 종류

| 스케줄러 | 동작 | 특징 |
|---|---|---|
| **StepLR** | N 에폭마다 γ배 감소 | 간단, 예측 가능 |
| **CosineAnnealingLR** | 코사인 곡선으로 감소 | 부드러운 감소, 최종 lr=0 |
| **ReduceLROnPlateau** | 검증 손실 정체 시 감소 | 자동 적응, 실무 많이 사용 |

In [ ]:
# 학습률 스케줄 시각화
dummy_model = nn.Linear(1, 1)
EPOCHS_VIZ  = 60
LR_INIT     = 0.1

schedulers = {
    'StepLR (step=15, γ=0.5)': lambda opt: torch.optim.lr_scheduler.StepLR(
        opt, step_size=15, gamma=0.5),
    'CosineAnnealingLR (T=60)': lambda opt: torch.optim.lr_scheduler.CosineAnnealingLR(
        opt, T_max=EPOCHS_VIZ, eta_min=1e-4),
    'CosineAnnealingWarmRestarts': lambda opt: torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
        opt, T_0=20, T_mult=1, eta_min=1e-4),
    'ExponentialLR (γ=0.95)': lambda opt: torch.optim.lr_scheduler.ExponentialLR(
        opt, gamma=0.95),
}
colors_sch = ['steelblue', 'tomato', 'seagreen', 'purple']

fig, ax = plt.subplots(figsize=(10, 4))
for (name, sched_fn), color in zip(schedulers.items(), colors_sch):
    opt  = torch.optim.SGD(dummy_model.parameters(), lr=LR_INIT)
    sch  = sched_fn(opt)
    lrs  = []
    for _ in range(EPOCHS_VIZ):
        lrs.append(opt.param_groups[0]['lr'])
        opt.step()
        sch.step()
    ax.plot(lrs, color=color, lw=2.5, label=name)

ax.set_title('학습률 스케줄러 비교 (초기 lr=0.1)')
ax.set_xlabel('Epoch')
ax.set_ylabel('Learning Rate')
ax.legend(fontsize=9)
plt.tight_layout()
plt.show()

### 3-2. 조기 종료 (Early Stopping)

검증 손실이 일정 에폭 동안 개선되지 않으면 학습을 중단합니다:

```
best_val_loss = ∞
patience = 10    ← 몇 에폭 더 기다릴지
counter  = 0

for each epoch:
    val_loss = validate()
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
        save_best_model()
    else:
        counter += 1
        if counter >= patience:
            stop training
```

In [ ]:
class EarlyStopping:
    """검증 손실 기반 조기 종료"""
    def __init__(self, patience=10, min_delta=1e-4):
        self.patience   = patience
        self.min_delta  = min_delta
        self.counter    = 0
        self.best_loss  = float('inf')
        self.best_state = None
        self.stopped_at = None

    def step(self, val_loss, model):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss  = val_loss
            self.counter    = 0
            self.best_state = {k: v.clone() for k, v in model.state_dict().items()}
            return False   # 계속 학습
        else:
            self.counter += 1
            return self.counter >= self.patience   # True = 종료

    def restore(self, model):
        if self.best_state:
            model.load_state_dict(self.best_state)


# 조기 종료 + 스케줄러 조합 학습
torch.manual_seed(42)
model_es   = make_classifier(use_dropout=True, use_bn=True)
opt_es     = torch.optim.Adam(model_es.parameters(), lr=0.005)
sched_es   = torch.optim.lr_scheduler.CosineAnnealingLR(opt_es, T_max=100, eta_min=1e-5)
crit_es    = nn.CrossEntropyLoss()
stopper    = EarlyStopping(patience=12)

# 검증용 로더 (Train의 20%)
n_val   = int(len(X_mn_tr) * 0.2)
X_val_e = X_mn_tr[-n_val:];  y_val_e = y_mn_tr[-n_val:]
X_tr_e  = X_mn_tr[:-n_val];  y_tr_e  = y_mn_tr[:-n_val]
val_loader_e  = make_loader(X_val_e, y_val_e, shuffle=False)
train_loader_e= make_loader(X_tr_e,  y_tr_e)

tr_losses_es, val_losses_es, lrs_es = [], [], []
stopped_epoch = None

for epoch in range(1, 150):
    # 학습
    model_es.train()
    ep_loss = 0
    for xb, yb in train_loader_e:
        opt_es.zero_grad()
        loss = crit_es(model_es(xb), yb)
        loss.backward()
        opt_es.step()
        ep_loss += loss.item() * len(xb)
    tr_losses_es.append(ep_loss / len(train_loader_e.dataset))

    # 검증
    val_loss, _ = evaluate(model_es, val_loader_e)
    val_losses_es.append(val_loss)
    lrs_es.append(opt_es.param_groups[0]['lr'])

    sched_es.step()
    if stopper.step(val_loss, model_es):
        stopped_epoch = epoch
        break

stopper.restore(model_es)   # 최적 가중치 복원

_, final_acc = evaluate(model_es, test_loader)
print(f'조기 종료: epoch {stopped_epoch} (best_val_loss={stopper.best_loss:.4f})')
print(f'최적 가중치 복원 후 Test Acc: {final_acc:.4f}')

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(13, 4))

epochs_range = list(range(1, len(tr_losses_es) + 1))
axes[0].plot(epochs_range, tr_losses_es,  color='steelblue', lw=2, label='Train Loss')
axes[0].plot(epochs_range, val_losses_es, color='tomato',    lw=2, label='Val Loss')
if stopped_epoch:
    best_e = np.argmin(val_losses_es) + 1
    axes[0].axvline(best_e,      color='seagreen', lw=2, linestyle='--',
                    label=f'최적 에폭={best_e}')
    axes[0].axvline(stopped_epoch, color='gray',   lw=2, linestyle=':',
                    label=f'종료 에폭={stopped_epoch}')
axes[0].set_title('조기 종료 학습 곡선')
axes[0].set_xlabel('Epoch'); axes[0].set_ylabel('Loss')
axes[0].legend(fontsize=9)

axes[1].plot(epochs_range, lrs_es, color='purple', lw=2)
axes[1].set_title('CosineAnnealingLR 학습률 변화')
axes[1].set_xlabel('Epoch'); axes[1].set_ylabel('Learning Rate')
axes[1].set_yscale('log')

plt.suptitle('CosineAnnealingLR + EarlyStopping 조합', y=1.02)
plt.tight_layout()
plt.show()

---
## Part 4. 다중 클래스 ROC · Label Smoothing

### 4-1. 다중 클래스 ROC (One-vs-Rest)

이진 ROC를 다중 클래스로 확장하는 **OvR (One-vs-Rest)** 전략:

$$\text{클래스 } k \text{ ROC} = \text{class } k \text{ vs 나머지 전체를 이진 문제로}$$

- **Macro AUC**: 클래스별 AUC 단순 평균 → 클래스 불균형 반영 안 함
- **Weighted AUC**: 샘플 수 기반 가중 평균 → 실제 분포 반영

In [ ]:
# MNIST 서브셋 (0~4 클래스만, 시각화 단순화)
mask_5 = y_mn_te < 5
X_5 = X_mn_te[mask_5]
y_5 = y_mn_te[mask_5]

torch.manual_seed(42)
model_roc = make_classifier(use_dropout=True, use_bn=True)

# 5-클래스 분류기로 재학습
mask_5_tr = y_mn_tr < 5
X_5_tr, y_5_tr = X_mn_tr[mask_5_tr], y_mn_tr[mask_5_tr]

# 출력 뉴런 5개로 교체
model_roc5 = nn.Sequential(
    nn.Linear(784, 256), nn.BatchNorm1d(256), nn.ReLU(), nn.Dropout(0.3),
    nn.Linear(256, 128), nn.BatchNorm1d(128), nn.ReLU(), nn.Dropout(0.3),
    nn.Linear(128, 5)
)
opt_roc = torch.optim.Adam(model_roc5.parameters(), lr=0.001)
for _ in range(20):
    model_roc5.train()
    for xb, yb in make_loader(X_5_tr, y_5_tr):
        opt_roc.zero_grad()
        loss = nn.CrossEntropyLoss()(model_roc5(xb), yb)
        loss.backward()
        opt_roc.step()

model_roc5.eval()
with torch.no_grad():
    probs_5 = F.softmax(model_roc5(X_5), dim=1).numpy()
y_5_np = y_5.numpy()

# OvR ROC 곡선
y_5_bin = label_binarize(y_5_np, classes=list(range(5)))
colors_roc = ['steelblue', 'tomato', 'seagreen', 'purple', 'orange']

fig, axes = plt.subplots(1, 2, figsize=(13, 5))
auc_per_class = []
for c, color in enumerate(colors_roc):
    fpr, tpr, _ = roc_curve(y_5_bin[:, c], probs_5[:, c])
    auc_c = roc_auc_score(y_5_bin[:, c], probs_5[:, c])
    auc_per_class.append(auc_c)
    axes[0].plot(fpr, tpr, color=color, lw=2, label=f'클래스 {c} (AUC={auc_c:.3f})')

axes[0].plot([0,1],[0,1], 'k--', lw=1.5, label='랜덤 (AUC=0.5)')
axes[0].set_title('OvR ROC 곡선 — 클래스 0~4')
axes[0].set_xlabel('FPR'); axes[0].set_ylabel('TPR')
axes[0].legend(fontsize=8)

macro_auc    = np.mean(auc_per_class)
weighted_auc = roc_auc_score(y_5_bin, probs_5, average='weighted', multi_class='ovr')

bars = axes[1].bar(range(5), auc_per_class, color=colors_roc, edgecolor='k', alpha=0.85)
axes[1].axhline(macro_auc,    color='black',  lw=2, linestyle='--',
                label=f'Macro AUC={macro_auc:.3f}')
axes[1].axhline(weighted_auc, color='gray',   lw=2, linestyle=':',
                label=f'Weighted AUC={weighted_auc:.3f}')
for bar, v in zip(bars, auc_per_class):
    axes[1].text(bar.get_x()+bar.get_width()/2, v+0.003, f'{v:.3f}', ha='center', fontsize=9)
axes[1].set_xticks(range(5)); axes[1].set_xticklabels([f'클래스 {i}' for i in range(5)])
axes[1].set_title('클래스별 AUC')
axes[1].set_ylim(0.9, 1.01)
axes[1].legend(fontsize=9)

plt.tight_layout()
plt.show()

### 4-2. Label Smoothing

**Label Smoothing**은 원-핫 레이블을 부드럽게 만들어 **과신뢰(overconfidence)** 를 방지합니다:

$$y_{\text{smooth}} = (1-\epsilon)\cdot y_{\text{one-hot}} + \frac{\epsilon}{K}$$

- $\epsilon = 0.1$이면: 정답 클래스 확률 목표 = $0.9 + 0.01 = 0.91$, 나머지 = $0.01$
- 모델이 **"완전히 확신"** 하는 대신 약간의 불확실성을 유지

In [ ]:
# Label Smoothing 효과 시각화
K = 10
epsilons = [0.0, 0.05, 0.1, 0.2]
target_class = 3

fig, axes = plt.subplots(1, 4, figsize=(14, 3))
for ax, eps in zip(axes, epsilons):
    y_smooth = np.full(K, eps / K)
    y_smooth[target_class] += (1 - eps)
    bars = ax.bar(range(K), y_smooth,
                  color=['tomato' if i == target_class else 'steelblue' for i in range(K)],
                  edgecolor='k', alpha=0.85)
    ax.set_ylim(0, 1.1)
    ax.set_title(f'ε={eps}\n정답 클래스 목표={y_smooth[target_class]:.2f}')
    ax.set_xticks(range(K))
    ax.set_xlabel('클래스')
    if eps == 0: ax.set_ylabel('레이블 값')
plt.suptitle(f'Label Smoothing — 정답 클래스={target_class} (빨강)', y=1.05)
plt.tight_layout()
plt.show()

# PyTorch CrossEntropyLoss에 label_smoothing 파라미터 내장
logits_demo = torch.tensor([[5.0, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]])
target_demo = torch.tensor([0])
print(f'Softmax 확률: {F.softmax(logits_demo, dim=1).detach().numpy().round(3)}')
for eps in [0.0, 0.1, 0.2]:
    loss = nn.CrossEntropyLoss(label_smoothing=eps)(logits_demo, target_demo)
    print(f'  label_smoothing={eps:.1f}: Loss={loss.item():.4f}')
print('→ Label Smoothing이 클수록 과신뢰 모델의 손실이 더 높아짐 (페널티)')

In [ ]:
# Label Smoothing 유무 학습 비교
def train_with_ls(label_smoothing, epochs=25):
    torch.manual_seed(42)
    m    = make_classifier(use_dropout=True, use_bn=True)
    opt  = torch.optim.Adam(m.parameters(), lr=0.001)
    crit = nn.CrossEntropyLoss(label_smoothing=label_smoothing)
    te_accs = []
    for _ in range(epochs):
        m.train()
        for xb, yb in train_loader:
            opt.zero_grad()
            crit(m(xb), yb).backward()
            opt.step()
        m.eval()
        with torch.no_grad():
            preds = m(X_mn_te).argmax(1)
        te_accs.append((preds == y_mn_te).float().mean().item())
    # 최종 예측 확신도 (정답 클래스 확률 평균)
    m.eval()
    with torch.no_grad():
        probs_all = F.softmax(m(X_mn_te), dim=1)
    confidence = probs_all.max(1).values.mean().item()
    return m, te_accs, confidence

ls_results = {}
for eps in [0.0, 0.1, 0.2]:
    m, accs, conf = train_with_ls(eps)
    ls_results[eps] = dict(model=m, accs=accs, confidence=conf)
    print(f'label_smoothing={eps:.1f}: 최종 Test Acc={accs[-1]:.4f}, 평균 확신도={conf:.4f}')

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
colors_ls = ['steelblue', 'tomato', 'seagreen']
for (eps, res), color in zip(ls_results.items(), colors_ls):
    axes[0].plot(res['accs'], color=color, lw=2, label=f'ε={eps}')
axes[0].set_title('Label Smoothing별 Test Accuracy'); axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy'); axes[0].legend()

confs = [res['confidence'] for res in ls_results.values()]
axes[1].bar([f'ε={e}' for e in ls_results.keys()], confs,
            color=colors_ls, edgecolor='k', alpha=0.85)
for i, v in enumerate(confs):
    axes[1].text(i, v+0.003, f'{v:.4f}', ha='center')
axes[1].set_title('평균 예측 확신도\n(낮을수록 과신뢰 적음)')
axes[1].set_ylim(0.8, 1.0)
plt.tight_layout()
plt.show()

---
## Exercise

### Exercise 1. Dropout 비율 체계적 실험

Dropout 비율 `p = 0.0, 0.1, 0.3, 0.5, 0.7`에 대해 MNIST 분류기를 각각 학습하고:  
1. Train Accuracy vs Test Accuracy 갭 (= 과적합 정도) 을 그래프로 시각화하세요.  
2. 리프 수 대신 **파라미터 수는 동일**하게 유지하면서 최적 Dropout 비율을 찾으세요.  
3. 어느 비율에서 Train-Test 갭이 가장 작으면서 Test Acc가 가장 높은지 출력하세요.

In [ ]:
# Exercise 1: Dropout 비율 실험
# Your code here

### Exercise 2. 불균형 MNIST — 두 가지 전략 성능 분석

클래스 0~4는 전체, 클래스 5~9는 **5%만** 사용하는 더 심한 불균형 상황에서:  
1. **Weighted CrossEntropyLoss** 와 **WeightedRandomSampler** 두 전략을 비교하세요.
2. 단순 Accuracy 외에 **클래스별 Precision / Recall**도 함께 출력하세요.  
3. 소수 클래스(5~9)에 대한 **Macro Recall** 기준 어떤 방법이 더 효과적인지 분석하세요.

In [ ]:
# Exercise 2: 심한 불균형 (5%) 처리 비교
# Your code here

### Exercise 3. (도전) ReduceLROnPlateau + EarlyStopping 파이프라인

다음 조건으로 학습 파이프라인을 완성하세요:
- 모델: `784 → 512 → 256 → 128 → 10` + BatchNorm + Dropout(p=0.4)
- 옵티마이저: `Adam`, 초기 lr=0.003
- 스케줄러: `ReduceLROnPlateau(mode='min', factor=0.5, patience=5)`
- 조기 종료: `patience=15`
- 학습 곡선(train/val loss)과 학습률 변화를 **하나의 그래프**에 겹쳐 시각화  
  (y축 왼쪽=Loss, y축 오른쪽=LR)
- 최종 Test Accuracy와 조기 종료된 에폭을 출력

In [ ]:
# Exercise 3: ReduceLROnPlateau + EarlyStopping
# Your code here

---
## Summary

| 개념 | 핵심 내용 |
|---|---|
| **Dropout** | 학습 중 뉴런 무작위 비활성화 → `model.train()` / `model.eval()` 구분 필수 |
| **BatchNorm** | 배치 내 활성화 정규화 → 학습 안정화, 빠른 수렴 |
| **Inverted Dropout** | train 시 $1/(1-p)$ 스케일링으로 eval 기댓값 보존 |
| **클래스 불균형** | Accuracy 역설 주의 — Macro F1 / Recall로 평가 |
| **Weighted Loss** | $w_c = N/(K \cdot n_c)$ — 소수 클래스 손실 가중 |
| **WeightedRandomSampler** | 소수 클래스 오버샘플링 — 에폭당 클래스 균형 유지 |
| **StepLR** | 고정 주기로 lr 감소 |
| **CosineAnnealing** | 코사인 곡선으로 부드럽게 lr 감소 |
| **ReduceLROnPlateau** | 검증 손실 정체 시 자동 lr 감소 |
| **Early Stopping** | 검증 손실 개선 없으면 학습 중단 + 최적 가중치 복원 |
| **OvR ROC** | 다중 클래스를 이진 문제로 분해해 클래스별 AUC 계산 |
| **Label Smoothing** | 원-핫 레이블 softening → 과신뢰 방지, 일반화↑ |

---

**다음 강의 (Week 7):** Tree-based Models — 결정 트리, 랜덤 포레스트, 앙상블